# String Algorithms Scratchpad

Exploring fuzzy matching and String Similarity tools to align datasets

#### Imports

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display, HTML

/Users/sm/.pyenv/versions/3.10.0/lib/python3.10/site-packages/pandas/compat/__init__.py:124: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)
pd.set_option('max_seq_item', None)

In [103]:
from difflib import SequenceMatcher
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
from thefuzz import fuzz
from thefuzz import process
import textdistance
import jaro
import jellyfish

----

##### Reading Dataframes

In [3]:
df = pd.read_csv("..//data//processed//stock_watchers_w_yfinance_03_12_2022.csv", encoding="utf-8")

In [4]:
df.head(5)

,transaction_date,disclosure_date,politician,owner,ticker,amount,asset_description,asset_type,transaction_type,comment,ptr_link,location,cap_gains,amount_low,amount_high,ticker2,name,sector,industry,longbusinesssummary,website,stock_description
0,02/24/2022,03/11/2022,Shelley M Capito,Spouse,NEE,1001 - 15000,"NextEra Energy, Inc. Common Stock",Stock,Sale (Partial),--,https://efdsearch.senate.gov/search/view/ptr/e7893c34-0761-4c2b-ac52-e303f166517f/,NaN,NaN,1001,15000.0,NEE,"NextEra Energy, Inc.",Utilities,Utilities—Regulated Electric,"NextEra Energy, Inc., through its subsidiaries, generates, transmits, distributes, and sells electric power to retail and wholesale customers in North America. The company generates electricity through wind, solar, nuclear, and fossil fuel, such as coal and natural gas facilities. It also develops, constructs, and operates long-term contracted assets with a focus on renewable generation facilities, electric transmission facilities, and battery storage projects; and owns, develops, constructs, manages and operates electric generation facilities in wholesale energy markets. As of December 31, 2020, the company operated approximately 28,400 megawatts of net generating capacity. It serves approximately 11 million people through approximately 5.6 million customer accounts in the east and lower west coasts of Florida with approximately 76,200 circuit miles of transmission and distribution lines and 673 substations. The company was formerly known as FPL Group, Inc. and changed its name to NextEra Energy, Inc. in 2010. NextEra Energy, Inc. was founded in 1925 and is headquartered in Juno Beach, Florida.",https://www.nexteraenergy.com,"Utilities, Utilities—Regulated Electric, NextEra Energy, Inc., through its subsidiaries, generates, transmits, distributes, and sells electric power to retail and wholesale customers in North America. The company generates electricity through wind, solar, nuclear, and fossil fuel, such as coal and natural gas facilities. It also develops, constructs, and operates long-term contracted assets with a focus on renewable generation facilities, electric transmission facilities, and battery storage projects; and owns, develops, constructs, manages and operates electric generation facilities in wholesale energy markets. As of December 31, 2020, the company operated approximately 28,400 megawatts of net generating capacity. It serves approximately 11 million people through approximately 5.6 million customer accounts in the east and lower west coasts of Florida with approximately 76,200 circuit miles of transmission and distribution lines and 673 substations. The company was formerly known as FPL Group, Inc. and changed its name to NextEra Energy, Inc. in 2010. NextEra Energy, Inc. was founded in 1925 and is headquartered in Juno Beach, Florida."
1,01/14/2022,02/14/2022,Thomas H Tuberville,Joint,NEE,15001 - 50000,"NextEra Energy, Inc. Common Stock",Stock,Sale (Full),--,https://efdsearch.senate.gov/search/view/ptr/c9da6bea-fa14-4a3a-9d8b-1745e834da59/,NaN,NaN,15001,50000.0,NEE,"NextEra Energy, Inc.",Utilities,Utilities—Regulated Electric,"NextEra Energy, Inc., through its subsidiaries, generates, transmits, distributes, and sells electric power to retail and wholesale customers in North America. The company generates electricity through wind, solar, nuclear, and fossil fuel, such as coal and natural gas facilities. It also develops, constructs, and operates long-term contracted assets with a focus on renewable generation facilities, electric transmission facilities, and battery storage projects; and owns, develops, constructs, manages and operates electric generation facilities in wholesale energy markets. As of December 31, 2020, the company operated approximately 28,400 megawatts of net generating capacity. It serves approximately 11 million people through approximately 5.6 million customer accounts in the east and lower west coasts of Florida with approximately 76,200 circuit miles

In [5]:
# df_subcomittees = pd.read_csv("..//data//processed//stock_watchers_w_yfinance_03_12_2022.csv", encoding="utf-8")

In [ ]:
# df3.to_csv('..//data//processed//stock_watchers_w_yfinance_03_12_2022.csv', index = False)

-----

##### Understanding FuzzyWuzzy + The Fuzz

Source: 
https://chairnerd.seatgeek.com/fuzzywuzzy-fuzzy-string-matching-in-python/

https://github.com/seatgeek/thefuzz

https://itnext.io/string-similarity-the-basic-know-your-algorithms-guide-3de3d7346227

Simple Ratios

In [12]:
m = SequenceMatcher(None, "NEW YORK METS", "NEW YORK MEATS")
m.ratio()

0.9629629629629629

In [20]:
fuzz.ratio("NEW YORK METS", "NEW YORK MEATS")

96

In [21]:
fuzz.ratio("YANKEES", "NEW YORK YANKEES")

61

In [22]:
fuzz.ratio("NEW YORK METS", "NEW YORK YANKEES")

76

In [27]:
fuzz.ratio("YANKEES", "NEW YOR")

14

In [28]:
fuzz.ratio("YANKEES", "EW YORK")

29

In [31]:
fuzz.ratio("YANKEES", "YANKEES")

100

In [61]:
fuzz.ratio("this is a test", "this is a test!")

97

Partial Ratio

In [48]:
# A perfect Partial Match
fuzz.partial_ratio("YANKEES", "NEW YORK YANKEES")

100

In [33]:
fuzz.partial_ratio("NEW YORK METS", "NEW YORK YANKEES")

69

Token Sort Ratio

In [50]:
#without token sort
fuzz.ratio("New York Mets vs Atlanta Braves", "Atlanta Braves vs New York Mets")

45

In [51]:
#with token sort
fuzz.token_sort_ratio("New York Mets vs Atlanta Braves", "Atlanta Braves vs New York Mets")

100

In [62]:
fuzz.ratio("fuzzy wuzzy was a bear", "wuzzy fuzzy was a bear")

91

In [63]:
fuzz.token_sort_ratio("fuzzy wuzzy was a bear", "wuzzy fuzzy was a bear")

100

Token Set

In [52]:
s1 = "mariners vs angels"
s2 = "los angeles angels of anaheim at seattle mariners"

In [53]:
t0 = "angels mariners"
t1 = "angels mariners vs"
t2 = "angels mariners anaheim angeles at los of seattle"

In [41]:
# t0 = [SORTED_INTERSECTION]
# t1 = [SORTED_INTERSECTION] + [SORTED_REST_OF_STRING1]
# t2 = [SORTED_INTERSECTION] + [SORTED_REST_OF_STRING2]

In [54]:
fuzz.ratio(t0, t1)

91

In [55]:
fuzz.ratio(t0, t2)

47

In [45]:
fuzz.ratio(t1, t2)

51

In [46]:
fuzz.token_set_ratio("mariners vs angels", "los angeles angels of anaheim at seattle mariners")

91

In [47]:
fuzz.token_set_ratio("Sirhan, Sirhan", "Sirhan")

100

In [64]:
fuzz.token_sort_ratio("fuzzy was a bear", "fuzzy fuzzy was a bear")

84

In [65]:
fuzz.token_set_ratio("fuzzy was a bear", "fuzzy fuzzy was a bear")

100

Process?

In [66]:
choices = ["Atlanta Falcons", "New York Jets", "New York Giants", "Dallas Cowboys"]

In [67]:
process.extract("new york jets", choices, limit=2)

[('New York Jets', 100), ('New York Giants', 79)]

In [72]:
process.extract("new york jets", choices, limit=3)

[('New York Jets', 100), ('New York Giants', 79), ('Atlanta Falcons', 29)]

In [68]:
process.extractOne("cowboys", choices)

('Dallas Cowboys', 90)

In [77]:
#parameters to extractOne to make it use a specific scorer
# >>> process.extractOne("System of a down - Hypnotize - Heroin", songs)
#     ('/music/library/good/System of a Down/2005 - Hypnotize/01 - Attack.mp3', 86)
# >>> process.extractOne("System of a down - Hypnotize - Heroin", songs, scorer=fuzz.token_sort_ratio)
#     ("/music/library/good/System of a Down/2005 - Hypnotize/10 - She's Like Heroin.mp3", 61)

----

##### Types of Algorithms

Source: https://itnext.io/string-similarity-the-basic-know-your-algorithms-guide-3de3d7346227

Based on the properties of operations, string similarity algorithms can be classified into a bunch of domains. Let’s discuss a few of them,
Edit distance based: Algorithms falling under this category try to compute the number of operations needed to transforms one string to another. More the number of operations, less is the similarity between the two strings. One point to note, in this case, every index character of the string is given equal importance.

Token-based: In this category, the expected input is a set of tokens, rather than complete strings. The idea is to find the similar tokens in both sets. More the number of common tokens, more is the similarity between the sets. A string can be transformed into sets by splitting using a delimiter. This way, we can transform a sentence into tokens of words or n-grams characters. Note, here tokens of different length have equal importance.

Sequence-based: Here, the similarity is a factor of common sub-strings between the two strings. The algorithms, try to find the longest sequence which is present in both strings, the more of these sequences found, higher is the similarity score. Note, here combination of characters of same length have equal importance.

Hamming Distance (finding the places where the strings vary)

In [88]:
#the edit distance is 1 for only the difference being one letter different
textdistance.hamming('text', 'test')

1

In [89]:
#75% similar between text and test
textdistance.hamming.normalized_similarity('text', 'test')

0.75

In [86]:
textdistance.hamming('arrow', 'arow')

3

In [87]:
textdistance.hamming.normalized_similarity('arrow', 'arow')

0.4

Levenshtein Distance (finding the number of edits which will transform one string to another)

In [91]:
#number of edits it will take to transform one to the other
textdistance.levenshtein('arrow', 'arow')

1

In [92]:
textdistance.levenshtein.normalized_similarity('arrow', 'arow')

0.8

Jaro-Winkler 

"(1) they contain same characters, but within a certain distance from one another, and (2) the order of the matching characters is same."

In [107]:
# textdistance.jaro_winkler("mes", "messi")

In [106]:
# textdistance.jaro_winkler("crate", "crat")

In [109]:
# textdistance.jaro_winkler("crate", "atcr")

Jaccard Index (find the number of common tokens and divide it by the total number of unique tokens)

"We first tokenize the string by default space delimiter, to make words in the strings as tokens. Then we compute the similarity score." 

In [117]:
tokens_1 = "hello world".split()
tokens_2 = "world hello".split()

In [118]:
textdistance.jaccard(tokens_1 , tokens_2)

1.0

In [119]:
tokens_1 = "hello new world".split()
tokens_2 = "hello world".split()

In [120]:
textdistance.jaccard(tokens_1 , tokens_2)

0.6666666666666666

Sorensen-Dice 

"Falling under set similarity, the logic is to find the common tokens, and divide it by the total number of tokens present by combining both sets." 

In [121]:
tokens_1 = "hello world".split()
tokens_2 = "world hello".split()

In [122]:
textdistance.sorensen(tokens_1 , tokens_2)

1.0

In [123]:
tokens_1 = "hello new world".split()
tokens_2 = "hello world".split()

In [124]:
textdistance.sorensen(tokens_1 , tokens_2)

0.8

Ratcliff-Obershelp similarity

In [125]:
string1, string2 = "i am going home", "gone home"

In [126]:
textdistance.ratcliff_obershelp(string1, string2)

0.6666666666666666

In [128]:
string1, string2 = "helloworld", "worldhello"

In [129]:
textdistance.ratcliff_obershelp(string1, string2)

0.5

In [130]:
string1, string2 = "test", "text"

In [131]:
textdistance.ratcliff_obershelp(string1, string2)

0.75

In [132]:
string1, string2 = "mes", "simes"

In [133]:
textdistance.ratcliff_obershelp(string1, string2)

0.75

In [134]:
string1, string2 = "arrow", "arow"

In [135]:
textdistance.ratcliff_obershelp(string1, string2)

0.8888888888888888

-----

Data Notes:

1. combine committee fullname with committee description in new column
2. remove duplicate words in each description
3. (agriculture vs. agricultural)

5. remove numbers and words, punctuation 

* a 
* includes
* deals 
* shall
* jurisdiction
* policy 
* member
* ranking
* 


